    차이점: 교수님 말씀대로 수동으로 다른 벤더의 값을 채우는게 아니라, 그냥 NaN으로 채우는 게 맞을 것 같다고 말씀하심

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [30]:
# Data Loading and pivot
data = pd.read_pickle("Data/cmktcap_data_final.pickle")

In [31]:
data.columns

Index(['coin_id', 'open', 'high', 'low', 'close', 'vol', 'mktcap', 'timehigh',
       'timelow'],
      dtype='object')

### 2. mktcap 음수 값 Check!

LUA 
ZNZ

In [4]:
data.loc[(data.index == "2021-05-30") & (data.coin_id == "4286"), "mktcap"] = np.nan
data.loc[(data.index == "2021-05-31") & (data.coin_id == "4286"), "mktcap"] = np.nan
data.loc[(data.index == "2021-06-01") & (data.coin_id == "4286"), "mktcap"] = np.nan

data.loc[(data.index == "2021-03-15") & (data.coin_id == "7216"), "mktcap"] = np.nan
data.loc[(data.index == "2021-03-16") & (data.coin_id == "7216"), "mktcap"] = np.nan
data.loc[(data.index == "2021-03-17") & (data.coin_id == "7216"), "mktcap"] = np.nan

점검

In [32]:
from multi_run_v3.data_pp import DataPreprocess

data_pp = DataPreprocess()
data_pp.initialize(data=data)
data_pp.make_mask(mktcap_thresh=1_000_000,
                  vol_thresh=1_000_000,
                  ma=True)

In [35]:
mktcap = (data_pp.mktcap_df * data_pp.mask)
weight = mktcap.apply(lambda x: x/ np.nansum(x), axis=1)
btc = weight.loc[:, '1']
eth = weight.loc[:, '1027']
xrp = weight.loc[:, '52']

In [63]:
tot = weight.loc[:,['1','1027']].loc["2014":].sum(1)

print(tot.loc[[tot.idxmax()]].round(3))
print(tot.loc[[tot.idxmin()]].round(3))
print(tot.iloc[[-1]].round(3))
print("Mean: ", tot.mean().round(3))

date
2015-05-04    0.984
dtype: float64
date
2018-01-07    0.485
dtype: float64
date
2023-04-16    0.707
dtype: float64
Mean:  0.789


In [52]:
btc_w = btc.loc['2014':].resample('Y').mean()
eth_w = eth.loc['2014':].resample('Y').mean()
#xrp_w = xrp.loc['2014':].resample('Y').mean()
total = weight.loc[:,['1','1027']].loc["2014":].sum(1).resample('Y').mean()

cc = pd.concat([btc_w, eth_w, total], 
          axis=1,
          keys=['BTC' ,'ETH', 'Total'])
cc.index = [str(cc)[:4] for cc in cc.index]
cc

,BTC,ETH,Total
2014,0.947259,NaN,0.947259
2015,0.948663,0.016955,0.949964
2016,0.883308,0.082984,0.961984
2017,0.619580,0.170277,0.789858
2018,0.476697,0.153881,0.630578
2019,0.632844,0.097304,0.730148
2020,0.662258,0.107452,0.769710
2021,0.506660,0.175744,0.682404
2022,0.460013,0.204207,0.664220
2023,0.477706,0.208951,0.686657


In [22]:
btc.loc['2014':].resample('Y').mean()

date
2014-12-31    0.947259
2015-12-31    0.948663
2016-12-31    0.883162
2017-12-31    0.618217
2018-12-31    0.471026
2019-12-31    0.619796
2020-12-31    0.636139
2021-12-31    0.481578
2022-12-31    0.400134
2023-12-31    0.418292
Freq: A-DEC, Name: 1, dtype: float64

### 3. BTC weight가 이상한 날을 잡아보자

10월 2일 / 10월 3일

7271 : Starname,IOV

In [14]:
data.loc[(data.index == "2021-10-02") & (data.coin_id == "7271"), "mktcap"] = np.nan
data.loc[(data.index == "2021-10-02") & (data.coin_id == "7271"), "close"] = np.nan

data.loc[(data.index == "2021-10-03") & (data.coin_id == "7271"), "mktcap"] = np.nan
data.loc[(data.index == "2021-10-03") & (data.coin_id == "7271"), "close"] = np.nan

data.loc[(data.index == "2021-10-04") & (data.coin_id == "7271"), "mktcap"] = np.nan
data.loc[(data.index == "2021-10-04") & (data.coin_id == "7271"), "close"] = np.nan

In [19]:
# Save the Cleaned Data
data.to_pickle("Data/cmktcap_data_final.pickle")